## 三、多输入通道和多输出通道
假设彩色图像的高和宽分别是 h 和 w（像素），那么它在内存中可以表示为一个 3×h×w 的多维数组。我们将大小为 3 的这一维称为通道（channel）维。
### 3.1 多输入通道
当输入数据含多个通道时，我们需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算。

假设输入数据的通道数为 $ c_i $，那么卷积核的输入通道数同样为 $ c_i $。设卷积核窗口形状为 $ k_h×k_w $。

当 $ c_i $=1 时，我们知道卷积核只包含一个形状为 $ k_h×k_w $ 的二维数组。

当 $ c_i $>1 时，我们将会为每个输入通道各分配一个形状为 $ k_h×k_w $ 的核数组。

把这 ci 个数组在输入通道维上连结，即得到一个形状为 $ c_i× k_h×k_w $ 的卷积核。

由于输入和卷积核各有 $ c_i $ 个通道，我们可以在各个通道上对输入的二维数组和卷积核的二维核数组做互相关运算，再将这 $ c_i $ 个互相关运算的二维输出按通道相加，得到一个二维数组。这就是含多个通道的输入数据与多输入通道的卷积核做二维相关运算的输出。

In [27]:
import gluonbook as gb
from mxnet import nd

def corr2d_multi_in(X,K):
    # 首先沿着 X 和 K 的第 0 维（通道维）遍历。然后使用 * 将结果列表变成 add_n 函数的位
    # 置参数（positional argument）来进行相加。
    for x,k in zip(X,K):
        print([gb.corr2d(x,k)])
    return nd.add_n(*[gb.corr2d(x,k) for x,k in zip(X,K)])

In [28]:
X = nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = nd.array([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

[
[[ 19.  25.]
 [ 37.  43.]]
<NDArray 2x2 @cpu(0)>]
[
[[ 37.  47.]
 [ 67.  77.]]
<NDArray 2x2 @cpu(0)>]



[[  56.   72.]
 [ 104.  120.]]
<NDArray 2x2 @cpu(0)>

### 3.2 多输出通道
当输入通道有多个时，由于我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是为 1。

设卷积核输入通道数和输出通道数分别为 $ c_i $ 和 $ c_o $，高和宽分别为 $ k_h $ 和 $ k_w $。

如果我们希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为 $ c_i×k_h×k_w $ 的核数组。

将它们在输出通道维上连结，卷积核的形状即 $ c_o×c_i×k_h×k_w $。

在互相关运算时，每个输出通道上的结果由卷积核在相同输出通道上的核数组与整个输入数组计算而来。

In [29]:
def corr2d_multi_in_out(X,K):
    # 对 K 的第 0 维遍历，每次同输入 X 做互相关计算。所有结果使用 stack 函数合并在一起。
    return nd.stack(*[corr2d_multi_in(X,k) for k in K])

In [30]:
K=nd.stack(K,K+1,K+2)
K.shape

(3, 2, 2, 2)

In [31]:
K


[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 1.  2.]
   [ 3.  4.]]]


 [[[ 1.  2.]
   [ 3.  4.]]

  [[ 2.  3.]
   [ 4.  5.]]]


 [[[ 2.  3.]
   [ 4.  5.]]

  [[ 3.  4.]
   [ 5.  6.]]]]
<NDArray 3x2x2x2 @cpu(0)>

In [32]:
corr2d_multi_in_out(X,K)

[
[[ 19.  25.]
 [ 37.  43.]]
<NDArray 2x2 @cpu(0)>]
[
[[ 37.  47.]
 [ 67.  77.]]
<NDArray 2x2 @cpu(0)>]
[
[[ 27.  37.]
 [ 57.  67.]]
<NDArray 2x2 @cpu(0)>]
[
[[  49.   63.]
 [  91.  105.]]
<NDArray 2x2 @cpu(0)>]
[
[[ 35.  49.]
 [ 77.  91.]]
<NDArray 2x2 @cpu(0)>]
[
[[  61.   79.]
 [ 115.  133.]]
<NDArray 2x2 @cpu(0)>]



[[[  56.   72.]
  [ 104.  120.]]

 [[  76.  100.]
  [ 148.  172.]]

 [[  96.  128.]
  [ 192.  224.]]]
<NDArray 3x2x2 @cpu(0)>

### 3.3 1*1卷积层
因为使用了最小窗口，1×1 卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。

实际上，1×1 卷积的主要计算发生在通道维上。

值得注意的是，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。

假设我们将通道维当做是特征维，将高和宽维度上的元素当成数据样本，那么 1×1 卷积层的作用与全连接层等价。

In [42]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]
    print("x,k:",X.shape,K.shape)
    X=X.reshape((c_i,h*w))
    K=K.reshape((c_o,c_i))
    print("x,k:",X.shape,K.shape)
    Y=nd.dot(K,X)
    print("y:",Y.shape)
    return Y.reshape((c_o,h,w))

In [43]:
X=nd.random.uniform(shape=(3,3,3))
K=nd.random.uniform(shape=(2,3,1,1))

Y1=corr2d_multi_in_out_1x1(X,K)
Y2=corr2d_multi_in_out(X,K)

(Y1-Y2).norm().asscalar() <1e-6

x,k: (3, 3, 3) (2, 3, 1, 1)
x,k: (3, 9) (2, 3)
y: (2, 9)
[
[[ 0.56538618  0.00934626  0.06686988]
 [ 0.4289299   0.5770601   0.46392322]
 [ 0.06617924  0.66934079  0.67244971]]
<NDArray 3x3 @cpu(0)>]
[
[[ 0.27925077  0.14902321  0.16205198]
 [ 0.31059361  0.01771634  0.1923307 ]
 [ 0.14346103  0.23507339  0.00635574]]
<NDArray 3x3 @cpu(0)>]
[
[[ 0.0345039   0.38891581  0.24900471]
 [ 0.86250246  0.10583018  0.31648219]
 [ 0.2607443   0.42341515  0.10453689]]
<NDArray 3x3 @cpu(0)>]
[
[[ 0.34010711  0.00562223  0.04022546]
 [ 0.25802207  0.34712955  0.27907225]
 [ 0.03981001  0.40264082  0.404511  ]]
<NDArray 3x3 @cpu(0)>]
[
[[ 0.80638844  0.43033215  0.46795517]
 [ 0.89689672  0.05115921  0.5553906 ]
 [ 0.4142704   0.67881805  0.01835339]]
<NDArray 3x3 @cpu(0)>]
[
[[ 0.0025138   0.02833465  0.01814136]
 [ 0.06283804  0.00771031  0.02305746]
 [ 0.01899665  0.03084812  0.00761609]]
<NDArray 3x3 @cpu(0)>]


True

### 3.4 小结
使用多通道可以拓展卷积层的模型参数。

假设我们将通道维当做是特征维，将高和宽维度上的元素当成数据样本，那么 1×1 卷积层的作用与全连接层等价。

1×1 卷积层通常用来调整网络层之间的通道数，并控制模型复杂度。